# Group Lab 3 – Comparative Urban Change in US States

Authors: Andrew Baker, Jamie Marken, James Lyou, Alberto Melendez, Emmanual Robi

## Part 1

Executive Summary:

The main objective of Lab 3 was to calculate and visualize urban change in WA State from 2008 to 2018. Definitions of "urban" and "non-urban" were used to determine urban change. First, to decide what makes a block group urban or non-urban, the density of census block groups in WA State were calculated. To determine the density, the US definition of urban was used which states that 1,000 people in 1 square mile is urban. This calculation proved that roughly 71% of census block groups in WA State are defined as urban in 2018. Then, the sum of urban land was divided by the sum of land in WA State to give the result of 2% of urbanized land in WA State in 2018. Next, the density of census block groups in 2008 were compared to 2018 block groups to determine a change in urban populations. This comparison indicates that census block groups in WA State are becoming more urban, as 65 block groups became urban from 2008 to 2018 while only 2 block groups became de-urbanized. Finally, two interactive maps were created. The first map visualizes which census block groups in WA State are de-urbanized, urbanized, or that experienced no change in urbanization in 2018. The second map visualizes census block groups that are classified as urban or non-urban in 2018.
 
For part 4, urbanization trends in Oregon and WA State were compared because they are in the same region and have similar land mass, although WA State has about 3 million more people. The methodology for Oregon was the same for WA State, but instead of using census block groups for WA State, counties were used for Oregon. The main result for urbanization classification in Oregon was that Multnomah County is the only county in Oregon that is classified as urban from 2012 to 2018. When the change in urban populations was calculated, the result showed that 0 counties experienced urban change from 2012 to 2018. Therefore, there were no changes in urban trends in Oregon from 2012 to 2018. The differences in urban trends in comparison to WA State are that 65 block groups became urban from 2008 to 2018, while 0 counties became urban in Oregon from 2012 to 2018. Another difference is that only one county (Multnomah) in Oregon is classified as urban, while 65 census block groups in WA State are classified as urban. 


Definitions of urban relative to the United States:

The United States splits "urban area" into two categories. 
Urbanized areas: indicates an area of 50,000 or more people. 
Urban clusters: indicates areas of at least 2,500 people and less than 50,000 people.

The United States defines census block groups as having a population density of at least 1,000 people per square mile.


Definitions of urban relative to Japan and New Zealand:

In Japan, urbanized areas are defined as bordering areas of densely populated districts. They number districts as units with a density requirement of 4,000 people per square km, or 10,000/ square mile.

New Zealand has three classes of urban areas.
main urban areas: the 17 urban areas which have a population of 30,000 more.
secondary urban areas: the 14 urban areas that have a population of 10,000 or more but less than 30,000.
minor urban areas: have a population of at least 1,000 but less than 10,000.

<a href="https://en.wikipedia.org/wiki/Urban_area">Link to Urban Area Reference</a>

Issues with US Census Data:

The data used below are Washington State census block data from Washington State’s Office of Financial Management. Potential issues with this data are how the US Census defines urban, non-urban, and suburban, which can be unclear. 

For example, the definitions of suburban are non-specific. The common definition of suburban is a mixed-use or residential area that is a part of a city/urban area or as a separate residential area within commuting distance to a city/urban area. Definitions of non-urban areas such as this can be messy, as it does not define a suburban area with a numerical density requirement.

<a href="https://en.wikipedia.org/wiki/Suburb">Link to Suburban Definition Reference</a>

There are issues with using US Census Data specifically with demographics. Regarding demographic analysis, there are limitations in the feasibility of producing estimates only at the national level, not at lower geographic levels. There are also issues with only recording broad racial categories (Black or non-Black). In addition, there is uncertainty in estimating total international migration to the United States, particularly emigration, temporary migration, and unauthorized migration.

<a href="https://www.census.gov/history/pdf/2010-background-crs.pdf">Link to Census Data Demographic Issues Reference</a>